In [1]:
import numpy as np
import pandas as pd
import json
import codecs
import datetime
import os
import sys


In [2]:
sys.path.append('../')

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (24, 16)

In [4]:
import oci
import json
from oci.config import validate_config


In [5]:
namespace = "lrhwvlagph9z"
bucket_name = "tenen-raw-data"
config = oci.config.from_file(
        "~/.apikey/config",
        "DEFAULT")

In [32]:
import ads
#from ads.common.auth import default_signer
#ads.set_auth(auth="api_key", profile="DEFAULT")
bucket_name = "tenen-raw-data"
folder_name = "products"
mon = 'dec'
tfile = f"tenen_hits_{mon}.csv"
#file_name = <file-name>
namespace = "lrhwvlagph9z"
df = pd.read_csv(f"oci://{bucket_name}@{namespace}/{folder_name}/{tfile}", storage_options={"config": "~/.apikey/config"},low_memory=False)
tfile

'tenen_hits_dec.csv'

In [33]:
df.shape

(1831008, 66)

In [34]:
df = df.replace(["", "/", ":", "(not set)", "UNKNOWN"], "0").fillna("0").astype(str)

In [17]:
connection_parameters = {
    "user_name": creds.get('user'),
    "password": creds.get('password'),
    "service_name": creds.get('sid'),
    "wallet_location": "/home/datascience/ADW/Wallet_DBTENENDATA.zip",
}

In [18]:
def check_col_types(df):
    x = df.isna().any()
    x.to_csv("/home/datascience/files/isna.csv")
    return dict(df.dtypes)

In [35]:
df = df.drop(columns=['hits.page.pagePath', 'hits.page.pageTitle', 
                      'hits.appInfo.screenName', 'hits.appInfo.landingScreenName',
                      'hits.appInfo.exitScreenName', 'hits.eventInfo.eventLabel',
                      'hits.page.pagePathLevel1', 'hits.page.pagePathLevel2',
                      'hits.page.pagePathLevel3', 'hits.page.pagePathLevel4'])

In [36]:
df.shape

(1831008, 56)

In [37]:
for c in df.columns:
    df[c] = df[c].str.slice(0,80)

In [38]:
df['fullVisitorId'].head(5)

0    1064989146145861665
1    1064989146145861665
2    1064989146145861665
3    1064989146145861665
4    1064989146145861665
Name: fullVisitorId, dtype: object

In [39]:
df.ads.to_sql(
    table_name='tenen_hit_list_dec',
    connection_parameters=connection_parameters,
    if_exists="replace"
)